## Import

In [1]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler, Adam

import torchvision.models as models
from torchvision import transforms
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, ShiftScaleRotate, Transpose, Compose, Normalize, Resize, RandomBrightnessContrast )
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from sklearn.model_selection import KFold

from efficientnet_pytorch import EfficientNet
import timm
from sklearn.model_selection import KFold, StratifiedKFold

import pickle


In [2]:
print(torch.__version__)

1.7.1+cu110


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


## Hyperparameters Setting

In [4]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':80,
    'LEARNING_RATE':1e-3,
    'freeze_lv': ('all',0),
    'BATCH_SIZE':32,
    'log': True,
    'SEED':41
}

In [5]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

## Fix RandomSeed

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [7]:
# 데이터 로드
with open('all_img_256.pkl', 'rb') as f:
    all_img_list = pickle.load(f)
with open('all_label_rm_CASE45-17.pkl', 'rb') as f:
    all_label = pickle.load(f)
with open('test_img_256.pkl', 'rb') as f:
    test_img_list = pickle.load(f)

In [8]:
if CFG['log']:
    all_label = list(np.log1p(all_label))

## CustomDataset

In [9]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        #image = cv2.imread(img_path)
        image = img_path
        if self.transforms is not None:
            image = self.transforms(image=image)['image'] 

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [10]:
################################################################
train_transform = Compose([
                      Transpose(p=0.5),
                      HorizontalFlip(p=0.5),
                      VerticalFlip(p=0.5),
                      ShiftScaleRotate(p=0.5),
                      RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                      Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
                      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                      ToTensorV2(p=1.0),
                    ], p=1.)

test_transform = Compose([
                      Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
                      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                      ToTensorV2(p=1.0),
                    ], p=1.)

## Define Model Architecture

In [11]:
def count_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        total_params+=param
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [12]:
class CNNRegressor(torch.nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.model = models.resnet50(pretrained=True)
        
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 1)
        
        count_parameters(self.model)
    def forward(self, inputs):
        output = self.model(inputs)
        return output

## Train

In [13]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device, fold):
    model.to(device)
    train_losses = []
    vali_losses = []
    test_losses = []
    # Loss Function
    criterion = nn.L1Loss().to(device) # L1Loss, SmoothL1Loss
    #criterion = NMAELoss().to(device)
    best_nmae = 9999
    best_mae = 9999
    scaler = torch.cuda.amp.GradScaler()
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        model.train()
        train_loss = []
        for img, label in (iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            # Calc loss
            loss = criterion(logit.squeeze(1), label)

            # backpropagation
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()  
            
            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
            
        # Evaluation Validation set
        vali_mae, vali_nmae = validation(model, vali_loader, criterion, device, best_mae)
        train_losses.append(np.mean(train_loss))
        vali_losses.append(vali_nmae)
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] \
        Validation MAE : [{vali_mae:.5f}] Validation NMAE : [{vali_nmae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './saved/res50_fold{}_shuffle.pth'.format(fold))
            print('Model Saved.')
            
    plt.plot(train_losses, label = 'trn')
    plt.plot(vali_losses, label = 'vali')
    plt.legend(loc='upper right')
    plt.show()

In [14]:
def validation(model, vali_loader, criterion, device, best_mae, test=False):
    model.eval() # Evaluation
    vali_loss = []
    model_pred = []
    label_true = []
    with torch.no_grad():
        for img, label in (iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())
            
            logit = logit.squeeze(1).detach().cpu()
            label = label.detach().cpu()
            label_true.extend(label.tolist())
            model_pred.extend(logit.tolist())
            
    vali_mae_loss = np.mean(vali_loss)
    if CFG['log']:
        vali_nmae_loss = NMAE(np.expm1(np.array(label_true)), np.expm1(np.array(model_pred)))
        diff = np.abs(np.expm1(np.array(label_true)) - np.expm1(np.array(model_pred)))
    else:
        vali_nmae_loss = NMAE(np.array(label_true), np.array(model_pred))
        diff = (np.array(label_true) - np.array(model_pred))
    
    
    if best_mae > vali_mae_loss:
        plt.figure(figsize=(30,2))
        plt.plot(diff)   
        plt.axhline(y=0.1*100, color='black')
        plt.axhline(y=0.2*100, color='black')
        plt.axhline(y=0.3*100, color='black')
        plt.axhline(y=0.4*100, color='black')
        plt.axhline(y=0.5*100, color='black')
        if CFG['log']:
            plt.axhline(y=np.expm1(np.mean(label_true)), color='red')
        else:
            plt.axhline(y=np.mean(label_true), color='red')
        ax_twin = plt.twinx()
        if CFG['log']:
            ax_twin.plot(np.expm1(np.array(label_true)), alpha=0.5)   
        else:
            ax_twin.plot(np.array(label_true), alpha=0.5)   
    plt.show()
    return vali_mae_loss, vali_nmae_loss

In [15]:
def validation(model, vali_loader, criterion, device, best_mae, test=False):
    model.eval() # Evaluation
    vali_loss = []
    model_pred = []
    label_true = []
    with torch.no_grad():
        for img, label in (iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())
            
            logit = logit.squeeze(1).detach().cpu()
            label = label.detach().cpu()
            label_true.extend(label.tolist())
            model_pred.extend(logit.tolist())
            
    vali_mae_loss = np.mean(vali_loss)
    if CFG['log']:
        vali_nmae_loss = NMAE(np.expm1(np.array(label_true)), np.expm1(np.array(model_pred)))
        diff = np.abs(np.expm1(np.array(label_true)) - np.expm1(np.array(model_pred)))
    else:
        vali_nmae_loss = NMAE(np.array(label_true), np.array(model_pred))
        diff = (np.array(label_true) - np.array(model_pred))
    
    
    if best_mae > vali_mae_loss:
        plt.figure(figsize=(30,2))
        plt.plot(diff)   
        plt.axhline(y=0.1*100, color='black')
        plt.axhline(y=0.2*100, color='black')
        plt.axhline(y=0.3*100, color='black')
        plt.axhline(y=0.4*100, color='black')
        plt.axhline(y=0.5*100, color='black')
        if CFG['log']:
            plt.axhline(y=np.expm1(np.mean(label_true)), color='red')
        else:
            plt.axhline(y=np.mean(label_true), color='red')
        ax_twin = plt.twinx()
        if CFG['log']:
            ax_twin.plot(np.expm1(np.array(label_true)), alpha=0.5)   
        else:
            ax_twin.plot(np.array(label_true), alpha=0.5)   
    plt.show()
    return vali_mae_loss, vali_nmae_loss

## Run!!

In [ ]:
list_idx = list(np.arange(0,len(all_img_list)))

kf = KFold(n_splits=5, shuffle=True, random_state=CFG['SEED'])
for fold, (train_idx, valid_idx) in enumerate(kf.split(list_idx)):
    print("#"*80)
    print("fold: {}".format(fold))
    train_img_list = [all_img_list[i] for i in train_idx]
    train_label = [all_label[i] for i in train_idx]

    vali_img_list = [all_img_list[i] for i in valid_idx]
    vali_label = [all_label[i] for i in valid_idx]
    
    # Get Dataloader
    train_dataset = CustomDataset(train_img_list, train_label, train_mode=True, transforms=train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    vali_dataset = CustomDataset(vali_img_list, vali_label, train_mode=True, transforms=test_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    model = CNNRegressor().to(device)

    optimizer = AdamW(model.parameters(), lr=CFG['LEARNING_RATE'])
    
    total_steps = int(len(train_dataset)*CFG['EPOCHS']/(CFG['BATCH_SIZE']))
    warmup_ratio = 0.1
    warmup_steps = int(total_steps * warmup_ratio) # warmup_steps = 1149
    print('total_steps: ', total_steps)
    print('warmup_steps: ', warmup_steps)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    train(model, optimizer, train_loader, vali_loader, scheduler, device, fold)

################################################################################
fold: 0
Total Trainable Params: 23510081


C:\Users\user\miniconda3\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


total_steps:  3180
warmup_steps:  318


## Inference

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

def predict_val(model, vali_loader, device):
    model.eval() # Evaluation
    vali_loss = []
    model_pred = []
    label_true = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            logit = logit.squeeze(1).detach().cpu()
            label = label.detach().cpu()
            label_true.extend(label.tolist())
            model_pred.extend(logit.tolist())
    return model_pred, label_true

In [ ]:

test_dataset = CustomDataset(test_img_list, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
pred_all = []
for fold in range(kf.get_n_splits(list_idx)):
    # Validation Score가 가장 뛰어난 모델을 불러옵니다.
    checkpoint = torch.load('./saved/res50_fold{}_shuffle.pth'.format(fold))
    model = CNNRegressor().to(device)

    model.load_state_dict(checkpoint)

    # Inference
    preds = predict(model, test_loader, device)
    if CFG['log']:
        preds = np.expm1(preds)
    pred_all.append(preds)
preds = np.array(pred_all).mean(axis=0)

## Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['leaf_weight'] = preds

submission.to_csv('./submission/submit_res50_5fold_shuffle.csv', index=False)